In [76]:
import pandas as pd
import numpy as np
import polars as pl

In [77]:
def payer_category(x):
    if 'Maximum Fee' in x:
        return 'max'
    elif 'Minimum Fee' in x:
        return'min'
    elif 'Cash Price' in x:
        return 'cash'
    elif 'Gross' in x:
        return 'gross'

In [78]:
folder = '.\\input_files\\'
file = '520608007_MedStar Franklin Square Medical Center_Standard Charges.xlsx'

df = pd.read_excel(folder + file, skiprows=3)

In [79]:
df.rename(columns={
    'Charge Description Master Detail': 'description',
    'REV/CPT/HCPCS': 'code',
    'Payor': 'payer',
    'Plan': 'plan',


}, inplace=True)

In [80]:
df.dropna(subset=['payer'], inplace=True)

In [81]:
cols = df.columns.tolist()
id_vars = cols[:4]
value_vars = ['Negotiated Fee Inpatient', 'Negotiated Fee Outpatient']
df_payers = pd.melt(df, id_vars=id_vars, value_vars=value_vars, var_name='payer_temp', value_name='standard_charge')

In [82]:
df_payers['setting'] = np.where(df_payers['payer_temp'].str.contains('Inpatient'), 'inpatient', np.where(df_payers['payer_temp'].str.contains('Outpatient'), 'outpatient', 1))
df_payers.drop('payer_temp', axis=1, inplace=True)

In [83]:
df_payers['payer_category'] = 'payer'

In [84]:
# Do non-payer columns
df.drop(['Negotiated Fee Inpatient', 'Negotiated Fee Outpatient', 'payer', 'plan'], axis=1, inplace=True)

In [85]:
cols = df.columns.tolist()
id_vars = cols[:2]
value_vars = cols[2:]

df = pd.melt(df, id_vars=id_vars, value_vars=value_vars, var_name='payer', value_name='standard_charge')

In [86]:
df['payer_category'] = df['payer'].apply(payer_category)
df['setting'] = np.where(df['payer'].str.contains('Inpatient'), 'inpatient', np.where(df['payer'].str.contains('Outpatient'), 'outpatient', 1))

In [87]:
df1 = pd.concat([df, df_payers])

In [88]:
df = df1

In [89]:
df.reset_index(drop=False, inplace=True)

In [90]:
df1 = df

In [106]:
df = df1

In [91]:
# df.drop(['level_0', 'index'], axis=1, inplace=True)

In [107]:
df.loc[df['code'].str.len() == 3, 'rev_code'] = df['code']
df.loc[df['code'].str.len() == 5, 'hcpcs_cpt'] = df['code']
df.loc[df['code'].str.len() == 7, 'hcpcs_cpt'] = df['code'].str[:5]
df.loc[df['code'].str.len() == 7, 'modifiers'] = df['code'].str[-2:]

In [ ]:
df.loc[df['rev_code'] == pd.NA]

In [108]:
df['rev_code'] = df['rev_code'].str.strip()
df['rev_code'] = df['rev_code'].apply(lambda x: str(x).zfill(4) if x else x)

In [109]:
df['code'].fillna('', inplace=True)
df['plan'].fillna('', inplace=True)
df['rev_code'].fillna('', inplace=True)
df['hcpcs_cpt'].fillna('', inplace=True)
df['modifiers'].fillna('', inplace=True)

In [110]:
ccn = {
    '460726303': '210062',
    '520491660': '210034',
    '520591607': '210056',
    '520591685': '210024',
    '520608007': '210015',
    '520619006': '210028',
    '520646893': '210018'
}

ein = file.split('_')[0]

df['hospital_id'] = ccn[ein]

df['code'] = df['code'].astype(str)

In [111]:
df2 = pl.from_pandas(df)

In [112]:
df2

index,description,code,payer,standard_charge,payer_category,setting,plan,rev_code,hcpcs_cpt,modifiers,hospital_id
i64,str,str,str,f64,str,str,str,str,str,str,str
0,"""MSG-PRIVATE R+…","""121""","""Gross Charge""",1830.0,"""gross""","""1""","""""","""0121""","""""","""""","""210015"""
1,"""MED SURG ROOM …","""121""","""Gross Charge""",1830.0,"""gross""","""1""","""""","""0121""","""""","""""","""210015"""
2,"""3SB ROOM AND B…","""121""","""Gross Charge""",1830.0,"""gross""","""1""","""""","""0121""","""""","""""","""210015"""
3,"""MED SURG ROOM …","""121""","""Gross Charge""",1830.0,"""gross""","""1""","""""","""0121""","""""","""""","""210015"""
4,"""MED SURG ROOM …","""121""","""Gross Charge""",1830.0,"""gross""","""1""","""""","""0121""","""""","""""","""210015"""
5,"""MS OVERFLOW RO…","""121""","""Gross Charge""",1830.0,"""gross""","""1""","""""","""0121""","""""","""""","""210015"""
6,"""MSG PRIVATE""","""121""","""Gross Charge""",1830.0,"""gross""","""1""","""""","""0121""","""""","""""","""210015"""
7,"""MED SURG ROOM …","""121""","""Gross Charge""",1830.0,"""gross""","""1""","""""","""0121""","""""","""""","""210015"""
8,"""MSG-PRIVATE R+…","""121""","""Gross Charge""",1830.0,"""gross""","""1""","""""","""0121""","""""","""""","""210015"""
